In [1]:
# %pip install datasets
# %pip install numpy
# %pip install matplotlib
# %pip install Pillow
# %pip install opencv-python
# %pip install torch

In [2]:
import os
from datasets import load_dataset
import random
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# class DatasetReceipt(Dataset):
#     def __init__(self, hf_datasets=None, local_folder=None, local_format="csv"):
#         """
#         Parameters:
#         - hf_datasets: list berisi nama dataset dari Hugging Face (string)
#         - local_folder: path ke folder yang berisi dataset lokal
#         - local_format: format dataset lokal, default "csv", bisa "parquet" atau lainnya
#         """
#         self.data = []

#         # Load dataset dari Hugging Face
#         if hf_datasets:
#             for dataset_name in hf_datasets:
#                 dataset = load_dataset(dataset_name, split="train")  # Load train split
#                 for item in dataset:
#                     self.data.append({"text": item["text"], "label": item.get("label", -1)})  # Sesuaikan kolomnya

#         # Load dataset dari folder lokal
#         if local_folder:
#             for file in os.listdir(local_folder):
#                 file_path = os.path.join(local_folder, file)
#                 if file.endswith(".csv") and local_format == "csv":
#                     df = pd.read_csv(file_path)
#                 elif file.endswith(".parquet") and local_format == "parquet":
#                     df = pd.read_parquet(file_path)
#                 else:
#                     continue

#                 for _, row in df.iterrows():
#                     self.data.append({"text": row["text"], "label": row.get("label", -1)})

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return self.data[idx]


In [ ]:
# dataset_international = load_dataset("katanaml-org/invoices-donut-data-v1")
# sample_image_international = dataset_international['train'][0]['image']
# print(sample_image_international)

# plt.imshow(sample_image_international)
# plt.axis("off")
# plt.show()

# mean = np.array([0.485])
# std = np.array([0.229])

# def resize_image(image):
#     """ Convert PIL image to NumPy array and resize """
#     if isinstance(image, Image.Image):  # Jika image berupa PIL.Image, konversi ke NumPy array
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("resize_image: Input harus berupa NumPy array")

#     return cv2.resize(image, (600, 600))

# def grey_image(image):
#     """ Convert image to grayscale """
#     if isinstance(image, Image.Image):
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("grey_image: Input harus berupa NumPy array")

#     return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# def normalize_image(image):
#     """ Normalize image using mean and std """
#     if isinstance(image, Image.Image):
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("normalize_image: Input harus berupa NumPy array")

#     image = image / 255.0
#     image = (image - mean) / std

#     return Image.fromarray((image * 255).astype(np.uint8))  # Kembalikan ke PIL image

# def preprocess(batch):
#     try:
#         batch['image'] = [resize_image(img) for img in batch['image']]
#         batch['image'] = [grey_image(img) for img in batch['image']]
#         # batch['image'] = [normalize_image(img) for img in batch['image']]

#         return batch
#     except Exception as e:
#         print(f"Error dalam preprocess: {e}")
#         return batch  # Supaya tidak crash

# dataset_international = dataset_international.map(preprocess, batched=True, num_proc=1)  # Gunakan num_proc=1 dulu

# sample_image_international_normalization = dataset_international['train'][0]['image']
# print(sample_image_international_normalization)

# plt.imshow(sample_image_international_normalization)
# plt.axis("off")
# plt.show()

In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch

class DatasetReceipt:
    def __init__(self, dataset_name="naver-clova-ix/cord-v2", split="train"):
        # Load dataset from Hugging Face
        self.dataset = load_dataset(dataset_name, split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample["image"]

        # Apply preprocessing
        image = self.resize_image(image)
        image = self.grey_image(image)
        # image = self.normalize_image(image)  # Uncomment if needed

        # Convert to tensor
        image_tensor = torch.tensor(image).unsqueeze(0)  # Add channel for grayscale

        return {"image": image_tensor, "label": sample.get("label", -1)}  # Default label to -1 if not present

    @staticmethod
    def resize_image(image):
        """Convert PIL image to NumPy array and resize"""
        if isinstance(image, Image.Image):  # If image is a PIL.Image, convert to NumPy array
            image = np.array(image)

        if not isinstance(image, np.ndarray):
            raise ValueError("resize_image: Input must be a NumPy array")

        return cv2.resize(image, (600, 600))

    @staticmethod
    def grey_image(image):
        """Convert image to grayscale"""
        if isinstance(image, Image.Image):
            image = np.array(image)

        if not isinstance(image, np.ndarray):
            raise ValueError("grey_image: Input must be a NumPy array")

        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    @staticmethod
    def normalize_image(image):
        """Normalize image using mean and std"""
        if isinstance(image, Image.Image):
            image = np.array(image)

        if not isinstance(image, np.ndarray):
            raise ValueError("normalize_image: Input must be a NumPy array")

        mean = np.array([0.4])
        std = np.array([0.2])
        image = image / 255.0
        image = (image - mean) / std

        return Image.fromarray((image * 255).astype(np.uint8))  # Convert back to PIL image

    def preprocess(self, batch):
        try:
            batch['image'] = [self.resize_image(img) for img in batch['image']]
            batch['image'] = [self.grey_image(img) for img in batch['image']]
            # batch['image'] = [self.normalize_image(img) for img in batch['image']]  # Uncomment if needed

            return batch
        except Exception as e:
            print(f"Error in preprocess: {e}")
            return batch  # Return batch to avoid crashing

In [ ]:
# Example usage
if __name__ == "__main__":
    # Load dataset
    dataset = load_dataset("naver-clova-ix/cord-v2")
    sample_image = dataset['train'][0]['image']
    print(sample_image)

    # Display sample image
    plt.imshow(sample_image)
    plt.axis("off")
    plt.show()

    # Create DatasetReceipt instance
    receipt_dataset = DatasetReceipt()

    # Preprocess the dataset
    dataset = dataset.map(receipt_dataset.preprocess, batched=True, num_proc=1)  # Use num_proc=1 for now

    # Display preprocessed sample image
    sample_image_normalization = dataset['train'][0]['image']
    print(sample_image_normalization)

    plt.imshow(sample_image_normalization, cmap='gray')
    plt.axis("off")
    plt.show()